In [1]:
import numpy as np
import scipy.sparse as sparse
from scipy.io import mmread
from scipy.linalg import expm, norm
from timeit import default_timer as timer

In [2]:
# download datasets
dol = mmread('dolphins.mtx').tocsr()
fb = mmread('facebook.mtx').tocsr()
pa = mmread('PA.mtx').tocsr()

In [3]:
# Arnoldi algorithm

# matrix A for which exp(A)b is of interest, n x n
# b initial vector to be used, length n
# m, the produced Krylov subspace will have dimension m

def arnoldi_iteration(A, m: int): 
    b  = np.ones(A.shape[0])  # b default to be 1_n
    
    n = A.shape[0]
    h = np.zeros((m + 1, m)) # to become the m x m upper Hessenberg matrix consisting of the coefficients h_ij
    V = np.zeros((n, m + 1)) # to become the orthonormal basis V_m = [v_1, v_2, ..., v_m]
    v = b / norm(b) # makes v a unit 2-norm vector    
    V[:, 0] = v # use v as the first Krylov vector
    
    for j in range(m):
        w = A @ v  # compute candidate vector
        
        for i in range(j + 1):
            h[i, j] = V[:,i] @ w # h_ij-th element is product of v_i and w
            w = w - h[i, j] * V[:, i] # modified Gram-Schmidt
            
        h[j + 1, j] = norm(w)
        
        zero = 1e-12 # small value used as h_ij = 0 threshold
        if h[j + 1, j] > zero: # if nonzero add v to the basis
            v = w / h[j + 1, j]
            V[:, i + 1] = v
        else: 
            return V, h 
        # print('step',j,'out of',m) # to check how far along algorithm is for larger m
    return V, h

In [4]:
# get Vm and Hm from output
def get_m(Vm1,Hm1,m):
    Vm, Hm = sparse.csc_matrix(Vm1[:,0:m]), sparse.csc_matrix(Hm1[0:m,0:m])
    return Vm, Hm

In [5]:
# get approximation from arnoldi result
def a_approximation(A, Vm, Hm, m):
    # get beta = ||v||_2
    b = np.ones(A.shape[0])
    beta = norm(b)

    # get unit vector
    e1 = np.zeros((m,1))
    e1[0] = 1
    e1 = sparse.csc_matrix(e1)
    
    X = beta * Vm @ expm(Hm) @ e1
    return X

In [6]:
########## TIMINGS

In [33]:
# dolphins

m = 100

start = timer()
for i in range(5):   # we run this 5 times to get more reliable timings
    dolV, dolH = arnoldi_iteration(dol, m)
end = timer()

dolTiming = (end - start)/(i+1)

print('this took', dolTiming, 'seconds', 'per run', i+ 1, 'runs') # Time in seconds

this took 0.016965699999997242 seconds per run 5 runs


In [34]:
# facebook

m = 100

start = timer()
for i in range(5):   # we run this 5 times to get more reliable timings
    fbV, fbH = arnoldi_iteration(fb, m)
end = timer()

fbTiming = (end - start)/(i+1)

print('this took', fbTiming, 'seconds', 'per run', i+ 1, 'runs') # Time in seconds

this took 0.2639195599999994 seconds per run 5 runs


In [35]:
# pennsylvania

m = 100

start = timer()
for i in range(5):   # we run this 5 times to get more reliable timings
    paV, paH = arnoldi_iteration(pa, m)
end = timer()

paTiming = (end - start)/(i+1)

print('this took', paTiming, 'seconds', 'per run', i+ 1, 'runs') # Time in seconds

this took 238.94743278000004 seconds per run 5 runs


In [37]:
start = timer()
V, H = get_m(paV,paH,m)
a_approximation(pa, V, H,m)
end = timer()
print(end-start)

62.927283299999544


In [ ]:
########## DOLPHIN ERROR PLOT

In [ ]:
########## FACEBOOK ERROR PLOT

In [ ]:
########## PA TIMINGS BAR CHART 